# TCAVの実行


このNotebookでは、TCAVの実行手順を説明します。以下が必要です。

1. **各フォルダーのサンプル画像の用意**
 * 概念画像
 * 調査したいクラス/ラベルの画像
 * CAVを学習するときの負例となるランダムな画像（どの概念にも属さないであろう画像）
2. **model wrapperの作成**：ModelWrapper抽象クラスのインスタンス（model.py内）.TCAVクラス（tcav.py）にモデルと通信する方法を指示する
3. **act_generatorの作成**：モデルからアクティベーションを取得する方法をTCAVクラスに伝えるActivationGeneratorInterfaceのインスタンス


## 要件
    pipでtcavとtensorflow（もしくはtensorflow-gpu）をインストール  

In [1]:
%load_ext autoreload
%autoreload 2
import os 

In [ ]:
import tcav.activation_generator as act_gen
import tcav.cav as cav
import tcav.model  as model
import tcav.tcav as tcav
import tcav.utils as utils
import tcav.utils_plot as utils_plot # utils_plot requires matplotlib
import tensorflow as tf

In [ ]:
tf.__version__

## ステップ1. 概念画像およびターゲットクラスの画像をディレクトリに保存する

画像を準備したら、パスを設定する。

**source_dir**：概念画像、ターゲットクラスの画像、ランダム画像が存在する場所  
それぞれ、source_dir内のサブフォルダである必要があります。  
ランダム画像ディレクトリは任意の名前にすることができます。 この例では、任意の理由で「random500_0」、「random500_1」を使用しています。  
概念画像とターゲットクラスごとに約50〜200枚の画像が必要です（10〜20枚の画像も動作する傾向がありますが、200枚はかなり安全です）。  


**cav_dir**：CAVを保存するディレクトリ（保存しない場合は `None`）  

**ターゲット、概念**：調査したいターゲットクラスの名前と概念（文字列）  
これらはsource_dirのフォルダー名です。

**ボトルネック**：TCAVに使用するボトルネック名（モデルの中間層）のリスト  
これらの名前は、モデルラッパーで定義されています。

In [ ]:
# 概念画像、ターゲットクラス、ランダム画像（CAVを学習する際の負のサンプル）が存在するディレクトリ
# それぞれがこのディレクトリ内のサブフォルダである必要があります。
# ランダム画像ディレクトリは任意の名前にすることができます。 この例では、任意の理由でrandom500_0、random500_1などを使用しています。
source_dir = '../datasets/for_tcav/'
activation_dir = '../activations'
# CAVを保存するディレクトリ（保存しない場合はなし）
cav_dir = '../cav'

utils.make_dir_if_not_exists(activation_dir)
utils.make_dir_if_not_exists(cav_dir)

# this is a regularizer penalty parameter for linear classifier to get CAVs. 
alphas = [0.1]   

# ターゲット、概念：ターゲットクラスの名前（調査する）および概念（文字列）-これらはsource_dirのフォルダー名です
concepts = ["black","red","yellow", 'blue', 'white', 'green']

## ステップ2. モデルラッパーを記述する

次のステップは、モデルと通信する方法をTCAVに伝えることです。詳細については、「model.GoolgeNetWrapper_public」を参照してください。  
https://github.com/tensorflow/tcav/blob/master/tcav/model.py#L333

ModelWrapper抽象クラスのサブクラスを定義できます。各関数が何をするか説明します（これらはかなり自明です）。  
このラッパーには、たとえば「get_prediction」など、すでに持っている多くの関数が含まれています。

### 1. Tensors from the graph: bottleneck tensors and ends
まず、ボトルネックテンソルを辞書として「self.bottlenecks_tensors」に保存します。 TCAVの実行に関心のあるボトルネックのみが必要です。同様に、「self.ends」辞書に「input」、「logit」、「prediction」テンソルを入力します。

### 2. 損失を定義する
損失テンソルを取得し、それを「self.loss」に割り当てます。これは、TCAVが方向導関数を取得するために使用するものです。

```python
self.y_input
```
これは、単にロジットレイヤーのターゲットインデックスのテンソルフロープレースホルダーです（たとえば、犬の場合はインデックス0、猫の場合はインデックス1）。
マルチクラス分類の場合、通常、次のように機能します。
```python
self.y_input = tf.placeholder（tf.int64、shape = [None]）
```

たとえば、マルチクラス分類子の場合、次のようなものが機能します。  

```python
    # Construct gradient ops.
    with g.as_default():
      self.y_input = tf.placeholder(tf.int64, shape=[None])

      self.pred = tf.expand_dims(self.ends['prediction'][0], 0)

      self.loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
              labels=tf.one_hot(self.y_input, len(self.labels)),
              logits=self.pred))
    self._make_gradient_tensors()
```

### 3.ラッパーの__init __（）で_make_gradient_tensorsを呼び出す
```python
_make_gradient_tensors（）
```
上記で定義された損失とボトルネックのテンソルが与えられると、勾配テンソルを追加します。

### 4.ラベル、画像の形、モデル名を入力します。
辞書形式のラベル（文字列）からロジットレイヤー（int）のインデックスへのマッピングを取得します。

```python
def id_to_label（self、idx）
def label_to_id（self、label）
```

入力画像の形状を「self.image_shape」に設定します

モデル名を「self.model_name」に設定します

モデルラッパーの作成はこれで完了です！InceptionV3とGooglenetの2つのモデルラッパーを作成しました。


**sess**: a tensorflow session.

In [ ]:
sess = utils.create_session()

GRAPH_PATH = '../models/model_op24/frozen_model.pb'
LABEL_PATH = '../datasets/for_tcav/label.txt'
    
mymodel = model.SimepleCNNWrapper_public(sess,
                                        GRAPH_PATH,
                                        LABEL_PATH)

## ステップ3. アクティベーションを返すクラスを実装

最後に、TCAVが特定の概念またはターゲットのサンプルデータをロードし、モデルラッパーを呼び出してアクティベーションを返すために使用するActivationGenerationInterfaceのクラスを実装します。このステップは多くの場合最も時間がかかるため、このロジックをmymodelの外部に引き出しました。モジュール化することで、アクティベーションをキャッシュしたり、計算を並列化したりできます。

アクティベーションジェネレータの `process_and_load_activations`メソッドは、最初のキーとして概念名またはターゲット名を持ち、2番目のキーとしてボトルネック名を持つアクティベーションの辞書を返す必要があります。：

```python
{concept1：{bottleneck1：[[0.2、0.1、....]]}、
concept2：{bottleneck1：[[0.1、0.02、....]]}、
target1：{bottleneck1：[[0.02、0.99、....]]}
```

In [ ]:
act_generator = act_gen.ImageActivationGenerator(mymodel, source_dir, activation_dir, max_examples=50)

## TCAV実行の準備完了

**num_random_exp**：意味のある概念の方向を確認するための実験の数。TCAVは、`random500_0`、` random500_1`などの名前のこのフォルダーを検索します。代わりに、`random_concepts`キーワードをランダムな概念のフォルダーのリストに設定することもできます。 有意義なテストを行うには、少なくとも10〜20を実行します。

**random_counterpart**：上記と同様に、オプションで統計テストの「ポジティブセット」としてランダム画像を含む単一のフォルダーを提供できます。 信頼性の低いランダムTCAVスコアを犠牲にして、計算時間を短縮します。

In [ ]:
mymodel.bottlenecks_tensors

In [ ]:
tf.logging.set_verbosity(0)
target = 'messon'  
# ボトルネック：TCAVに使用するボトルネック名（モデルの中間層）のリスト
# bottlenecks = ['conv2d_1', 'conv2d_3', 'conv2d_4', 'dense_2']
bottlenecks = ['activation_5']

num_random_exp = 3

## only running num_random_exp = 10 to save some time. The paper number are reported for 500 random runs. 
mytcav = tcav.TCAV(sess,
                   target,
                   concepts,
                   bottlenecks,
                   act_generator,
                   alphas,
                   cav_dir=cav_dir,
                   num_random_exp=num_random_exp)
print ('This may take a while... Go get coffee!')
results = mytcav.run(run_parallel=True)
print ('done!')

In [ ]:
utils_plot.plot_results(results, num_random_exp=num_random_exp)